In [10]:
import pandas as pd
import os
import datetime as dt
from tqdm import tqdm
import requests
import io



In [ ]:
df1 = pd.read_csv("./3618973.csv")
df2 = pd.read_csv("./3618766.csv")
df = pd.concat([df1,df2])

In [18]:
def date_fun(date):
    if date != date:
        return None
    date = date.split("-")
    return {"Year" : int(date[0]), "Month" : int(date[1]),"Day" : int(date[2])}
df[["Year","Month","Day"]] = df.apply(lambda x : date_fun(x["DATE"]),axis = 1,result_type = "expand")
df = df[df["Year"] >= 1965]
df = df[df["TMAX"].isna() != True]
df = df.dropna(axis=1,thresh = len(df)*.5)
df["Datetime"] = df.apply(lambda x : dt.datetime(x["Year"],x["Month"],x["Day"]),axis = 1)

def get_era5_data(year_start,year_end):
    key = "ade9eb48979c4310ae7b3ba195cfd999"
    key2 = "2b13ca426b96408d9d0c1f3814d15390"
    key0 = "a5c8c0da0c5f449fb21e280d3b6dbd3a"
    key3 = "bde9418ce268472293a3d35791409fbc"
    url = 'https://api.oikolab.com/weather'
    
    r = requests.get(url,
                     params={'param': ['temperature'],
                             'lat': [40.78,41.7,30.2,25.8],
                             'lon': [-74,-87.7,-97.7,80.3],
                             'location_id': ['NY','C','A','M'],
                             'freq' : 'D',
                             'start': str(year_start)'-01-01',
                             'end': str(year_end) + '-12-31'},
                     headers={'api-key': key3}
                     )
    a = ast.literal_eval(ast.literal_eval(r.text)["data"])
    a = pd.DataFrame(columns = a["columns"], data = a["data"])
    for i, row in a.iterrows():
        a.at[i,"datetime"] = dt.datetime(2000,1,1) + dt.timedelta(days = i%(len(a)/4))
    a.to_csv("forecasts"+(year_start)+"_"+(year_end)+".csv",index = False)

In [6]:
stat = None
track = []
for i,row in tqdm(df.iterrows()):
    if row["Year"] == 2024:
        continue
    if row["STATION"] != stat:
        track = []
        stat = row["STATION"]
    if len(track) < 30:
        p = row["PRCP"]
        track.append(row["TMAX"])
        continue
    df.at[i,"ry"] = p
    df.at[i,"Week_av"] = np.sum(track[-7:])/7
    df.at[i,"2Week_av"] = np.sum(track[-14:])/14
    df.at[i,"Month_av"] = np.sum(track)/30
    for j in range(1,31):
        df.at[i,"d" + str(j)] = track[30-j]
    track = track[1:]
    track.append(row["TMAX"]) 
    p = row["PRCP"]
        

84320it [21:58, 63.96it/s] 


In [27]:
nyurl = "https://mesonet.agron.iastate.edu/cgi-bin/request/daily.py?network=NY_ASOS&stations=NYC"
miaurl = "https://mesonet.agron.iastate.edu/cgi-bin/request/daily.py?network=FL_ASOS&stations=MIA"
chiurl = "https://mesonet.agron.iastate.edu/cgi-bin/request/daily.py?network=IL_ASOS&stations=MDW"
ausurl = "https://mesonet.agron.iastate.edu/cgi-bin/request/daily.py?network=TX_ASOS&stations=ATT"
df_hds = []
for city in ["M","C","A","NY"]:
    url = "&year1=1965&month1=1&day1=1&year2=2024&month2=3&day2=17&var=max_rh&na=blank&format=csv"
    u = {"NY" : nyurl,"M" : miaurl, "C" : chiurl,"A" : ausurl}[city] + url
    r = requests.get(u).text
    df_hd = pd.read_csv(io.StringIO(r),sep = ",")
    df_hd[["Year","Month","Day"]] = df_hd.apply(lambda x : date_fun(x["day"]),axis = 1,result_type = "expand")
    df_hd["STATION"] = df_hd["station"].replace({"MIA": 'USW00012839', "MDW":'USW00014819', "ATT":'USW00013904', "NYC":'USW00094728'})
    df_hd.drop(columns = ["station",'day'],inplace = True)
    df_hds.append(df_hd)
    #df_hd.apply(lambda x : {"MIA": 'USW00012839', "MDW":'USW00014819', "ATT":'USW00013904', "NYC":'USW00094728'}[x["station"]],axis = 1,result_type = "expand")
df_hds = pd.concat(df_hds)
df = pd.merge(df,df_hds,on = ['STATION', 'Year', 'Month', 'Day'],how = "inner")
    

In [44]:
dfs = []
for s in ["2000_2023","1970_1999","1965_1969"]:
    dfs.append(pd.read_csv("forecasts" + s + ".csv"))
    d = pd.read_csv("forecasts" + s + ".csv")
dfs = pd.concat(dfs)
dfs[["Year","Month","Day"]] = dfs.apply(lambda x : date_fun(x["datetime"]),axis = 1,result_type = "expand")
dfs["STATION"] = dfs["location_id (id)"].replace({"M": 'USW00012839', "C":'USW00014819', "A":'USW00013904', "NY":'USW00094728'})
dfs["temp_forecast"] = (dfs["temperature (degC)"] * 1.8) + 32
dfs = dfs[["Year","Month","Day","STATION","temp_forecast"]]
df = pd.merge(df,dfs,on = ['STATION', 'Year', 'Month', 'Day'],how = "inner")
df.to_csv("for_1965_2024.csv",index=False)

0
0
0
[2000 2001 2002 2003 2004 2005 2006 2007 2008 2009 2010 2011 2012 2013
 2014 2015 2016 2017 2018 2019 2020 2021 2022 2023 1970 1971 1972 1973
 1974 1975 1976 1977 1978 1979 1980 1981 1982 1983 1984 1985 1986 1987
 1988 1989 1990 1991 1992 1993 1994 1995 1996 1997 1998 1999 1965 1966
 1967 1968 1969]


In [7]:
import pandas as pd
df = pd.read_csv("for_1965_2024.csv")
df["decade"] = df.apply((lambda x : x["Year"] // 10 - 196),axis=1)

In [16]:
start = dt.datetime(1965,1,1)
df["Datetime"] = df.apply(lambda x : dt.datetime(x["Year"],x["Month"],x["Day"]),axis = 1)
df["time_diff"] = df.apply((lambda x : x["Datetime"] - start),axis = 1)

In [15]:
df

,STATION,NAME,DATE,AWND,PGTM,PRCP,SNOW,SNWD,TMAX,TMIN,...,d25,d26,d27,d28,d29,d30,max_rh,temp_forecast,decade,time_diff
0,USW00012839,"MIAMI INTERNATIONAL AIRPORT, FL US",1965-01-01,NaN,1612.0,0.23,0.0,0.0,78.0,70.0,...,74.0,75.0,83.0,83.0,81.0,76.0,81.001440,65.444,0,0 days
1,USW00012839,"MIAMI INTERNATIONAL AIRPORT, FL US",1965-01-02,NaN,42.0,0.00,0.0,0.0,77.0,63.0,...,73.0,74.0,75.0,83.0,83.0,81.0,84.260040,66.866,0,1 days
2,USW00012839,"MIAMI INTERNATIONAL AIRPORT, FL US",1965-01-03,NaN,2312.0,0.07,0.0,0.0,78.0,59.0,...,78.0,73.0,74.0,75.0,83.0,83.0,92.564200,60.998,0,2 days
3,USW00012839,"MIAMI INTERNATIONAL AIRPORT, FL US",1965-01-04,NaN,1048.0,0.06,0.0,0.0,73.0,64.0,...,78.0,78.0,73.0,74.0,75.0,83.0,93.275604,63.356,0,3 days
4,USW00012839,"MIAMI INTERNATIONAL AIRPORT, FL US",1965-01-05,NaN,1012.0,0.00,0.0,0.0,72.0,55.0,...,79.0,78.0,78.0,73.0,74.0,75.0,74.564210,61.088,0,4 days
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61005,USW00094728,"NY CITY CENTRAL PARK, NY US",2023-12-27,5.14,NaN,0.32,0.0,0.0,50.0,45.0,...,60.0,51.0,51.0,38.0,41.0,56.0,92.769646,44.366,6,21544 days
61006,USW00094728,"NY CITY CENTRAL PARK, NY US",2023-12-28,6.93,NaN,1.22,0.0,0.0,55.0,49.0,...,53.0,60.0,51.0,51.0,38.0,41.0,92.860740,49.622,6,21545 days
61007,USW00094728,"NY CITY CENTRAL PARK, NY US",2023-12-29,3.36,NaN,0.05,0.0,0.0,55.0,46.0,...,54.0,53.0,60.0,51.0,51.0,38.0,92.769646,48.614,6,21546 days
61008,USW00094728,"NY CITY CENTRAL PARK, NY US",2023-12-30,6.49,NaN,0.00,0.0,0.0,46.0,39.0,...,43.0,54.0,53.0,60.0,51.0,51.0,67.899820,43.718,6,21547 days


In [17]:
df.to_csv("for_1965_2024.csv",index=False)